This notebook will allow us to experiment with editing CSV files and hopefully eventually allow us to combine them. Python has a built-in CSV library we can use.

In [1]:
import csv
filename = "popular-games-no-reviews.csv"

# Starter code from https://www.geeksforgeeks.org/working-csv-files-python/#
# initializing the titles and rows list
original_fields = []
original_rows = []
 
# reading csv file
with open(filename, 'r', encoding='Latin1') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting field names through first row
    original_fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        # Only add if the game has been released
        if row[2] != "releases on TBD":
            original_rows.append(row)
 
    # get total number of rows
    print("Total no. of rows: %d" % (csvreader.line_num))
 
# printing the field names
print('Field names are:' + ', '.join(field for field in original_fields))

Total no. of rows: 1513
Field names are:, Title, Release Date, Team, Rating, Times Listed, Number of Reviews, Genres, Plays, Playing, Backlogs, Wishlist


So now we have the list of rows stored in the variable "rows".

In [2]:
print(original_rows[0:5])

[['0', 'Elden Ring', '25-Feb-22', "['Bandai Namco Entertainment', 'FromSoftware']", '4.5', '3.9K', '3.9K', "['Adventure', 'RPG']", '17K', '3.8K', '4.6K', '4.8K'], ['1', 'Hades', '10-Dec-19', "['Supergiant Games']", '4.3', '2.9K', '2.9K', "['Adventure', 'Brawler', 'Indie', 'RPG']", '21K', '3.2K', '6.3K', '3.6K'], ['2', 'The Legend of Zelda: Breath of the Wild', '3-Mar-17', "['Nintendo', 'Nintendo EPD Production Group No. 3']", '4.4', '4.3K', '4.3K', "['Adventure', 'RPG']", '30K', '2.5K', '5K', '2.6K'], ['3', 'Undertale', '15-Sep-15', "['tobyfox', '8-4']", '4.2', '3.5K', '3.5K', "['Adventure', 'Indie', 'RPG', 'Turn Based Strategy']", '28K', '679', '4.9K', '1.8K'], ['4', 'Hollow Knight', '24-Feb-17', "['Team Cherry']", '4.4', '3K', '3K', "['Adventure', 'Indie', 'Platform']", '21K', '2.4K', '8.3K', '2.3K']]


In [3]:
# These constants correspond to the index of the field
TITLE = 1
RELEASE_DATE = 2
TEAM = 3
RATING = 4
TIMES_LISTED = 5
NUMBER_OF_REVIEWS = 6
GENRES = 7
PLAYS = 8
PLAYING = 9
BACKLOGS = 10
WISHLIST = 11

print(original_rows[3][TITLE])
print(original_rows[3][GENRES])

Undertale
['Adventure', 'Indie', 'RPG', 'Turn Based Strategy']


Next, create the new fields that we want to add to. Don't run the cell directly below this more than once!

In [4]:
DEVELOPER = 12
PUBLISHER = 13

original_fields.append("Developer")
original_fields.append("Publisher")

In [5]:
# Read in VGChartz DB. Do it with a dictionary this time!
# initializing the titles and rows list
VGCHARTZ_NAME = 0
VGCHARTZ_DATE = 1
VGCHARTZ_PUBLISHER = 3
VGCHARTZ_DEVELOPER = 4
VGCHARTZ_SHIPPED = 5

vgchartz_fields = []
vgchartz_rows = []
vgchartz_dict = {}    # store TITLE -> INDEX, so that we can quickly find the correct row for any given game.
                      # We must use lists, because multiple games may be published under the same name (e.g. Tetris).

# reading csv file
with open("vgchartz.csv", 'r', encoding='Latin1') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting field names through first row
    vgchartz_fields = next(csvreader)
 
    # extracting each data row one by one
    index = 0
    for row in csvreader:
        vgchartz_rows.append(row)
        # Next, take care of the dictionary
        if row[VGCHARTZ_NAME] in vgchartz_dict:
            # If there already is a value here, append the new value to the list
            vgchartz_dict[row[VGCHARTZ_NAME]].append(index)
        else:
            # If there isn't a value here yet, create a list for the dictionary
            vgchartz_dict[row[VGCHARTZ_NAME]] = [index]
            
        index += 1
 
    # get total number of rows
    print("Total no. of rows: %d" % (csvreader.line_num))
 
# printing the field names
print('Field names are:' + ', '.join(field for field in vgchartz_fields))

Total no. of rows: 64006
Field names are:name, date, platform, publisher, developer, shipped, total, america, europe, japan, other, vgc, critic, user


In [6]:
# Test cases
print(vgchartz_dict["Tetris"]) # This means that rows 0, 92, 693, 49762, and 49763 contain versions of Tetris


[0, 92, 693, 49762, 49763]


Fill out the new fields. We can do that by looping through original_rows and extracting data from vgchartz_rows using vgchartz_dict for each entry.

In [7]:
RELEASE_YEAR = 14
RELEASE_MONTH = 15

original_fields.append("Year Released")
original_fields.append("Month Released")

for row in original_rows:
    # First, find the corresponding entry in vgchartz
    title = row[TITLE]
    date = row[RELEASE_DATE]
    true_year = date[-2:]    # in format e.g. "17" or "23"

    # Check that it exists
    if title in vgchartz_dict:
        # Once we get here, there is at least one entry in vgchartz for this title
        possible_vgchartz_indices = vgchartz_dict[title]
        pub = None
        dev = None

        # Pick out which of the possible indices is the correct one using the release date
        true_index = possible_vgchartz_indices
        found_info = False
        for index in possible_vgchartz_indices:
            possible_year = vgchartz_rows[index][VGCHARTZ_DATE][2:4]

            # If possible year is the same as true year, assume these are the same games. Also, don't overwrite, in case of repeat entries,
            # assume the higher index is the better one
            if possible_year == true_year and not found_info:
                pub = vgchartz_rows[index][VGCHARTZ_PUBLISHER]
                dev = vgchartz_rows[index][VGCHARTZ_DEVELOPER]
                # shipped = vgchartz_rows[index][VGCHARTZ_SHIPPED]
                found_info = True
    
        # Append the correct developer, publisher, shipped information as found from vgchartz
        row.append(dev)
        row.append(pub)
        # row.append(shipped)
        
    else:
        # There was no entry in vgchartz for this title, so we have no publisher/developer data
        row.append(None)    # TODO: this might not be the right syntax for an empty entry, double check this
        row.append(None)
        # row.append(None)

    # While we have the true year picked out, add release year and release month as their own columns
    if int(true_year) < 25:
        row.append("20" + true_year)
    else:
        row.append("19" + true_year)
    row.append(date.split("-")[1])

        

In [8]:
# Check that the above code worked by printing some examples
i = 0
while i < 5:  
    print(original_rows[i][TITLE])
    print(original_rows[i][TEAM])
    print(original_rows[i][DEVELOPER])
    print(original_rows[i][PUBLISHER])
    print()
    i += 1

Elden Ring
['Bandai Namco Entertainment', 'FromSoftware']
From Software
Bandai Namco Entertainment

Hades
['Supergiant Games']
None
None

The Legend of Zelda: Breath of the Wild
['Nintendo', 'Nintendo EPD Production Group No. 3']
Nintendo
Nintendo

Undertale
['tobyfox', '8-4']
Toby Fox
Toby Fox

Hollow Knight
['Team Cherry']
Team Cherry
Team Cherry



Now the entries of publisher and developer sometimes give redundant indoemation from the team entries, but sometimes they're written in a format that might be more helpful (e.g. Toby Fox vs. the stylized tobyfox).

Next, consider how to label indie vs. non-indie.

In [9]:
from enum import Enum

class Game_type(Enum):
    AAA = 1
    INDIE = 2
    UNKNOWN = 3


# Create a dictionary that maps from the publisher list to publishing type: String -> Game_type
publisher_type_dict = {}

# reading curated publisher list csv
with open("Seyed's Publisher Database - Publishers (curated).csv", 'r', encoding='Latin1') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting each data row one by one
    for row in csvreader:
        publisher_type_dict[row[0]] = Game_type.INDIE

# reading open publisher list csv
with open("Seyed's Publisher Database - Publishers (open).csv", 'r', encoding='Latin1') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting each data row one by one
    for row in csvreader:
        publisher_type_dict[row[0]] = Game_type.INDIE

# Add our own publisher list
with open("our-indie-publisher-list.csv", 'r', encoding='utf8') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting each data row one by one
    for row in csvreader:
        publisher_type_dict[row[0]] = Game_type.INDIE
        
# Remove the demarcations used in the Seyed DBs
publisher_type_dict.pop('')
publisher_type_dict.pop('Publisher')
publisher_type_dict.pop('HIGH BUDGET')
publisher_type_dict.pop('MID BUDGET')
publisher_type_dict.pop('LOW BUDGET')


# Now add the list of AAA publishers
# reading open publisher list csv
with open("aaa-aa-publisher-list.csv", 'r', encoding='Latin1') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting each data row one by one
    for row in csvreader:
        publisher_type_dict[row[0]] = Game_type.AAA

# Add our own publisher list
with open("our-aaa-publisher-list.csv", 'r', encoding='utf8') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting each data row one by one
    for row in csvreader:
        publisher_type_dict[row[0]] = Game_type.AAA

print(publisher_type_dict)

{'Coffee Stain': <Game_type.INDIE: 2>, 'Paradox': <Game_type.INDIE: 2>, 'Focus Entertainment': <Game_type.AAA: 1>, '505 Games': <Game_type.INDIE: 2>, 'Team17': <Game_type.AAA: 1>, 'Devolver': <Game_type.INDIE: 2>, 'Curve Games': <Game_type.INDIE: 2>, 'tinyBuild': <Game_type.INDIE: 2>, 'Humble Games': <Game_type.INDIE: 2>, 'Funcom': <Game_type.INDIE: 2>, 'Private Division': <Game_type.INDIE: 2>, '11 bit studios': <Game_type.INDIE: 2>, 'Square Enix Collective': <Game_type.INDIE: 2>, 'Behaviour Interactive': <Game_type.INDIE: 2>, 'Tripwire Interactive': <Game_type.INDIE: 2>, 'Annapurna Interactive': <Game_type.INDIE: 2>, 'PlayWay': <Game_type.INDIE: 2>, 'XSEED Games': <Game_type.INDIE: 2>, 'Starbreeze': <Game_type.INDIE: 2>, 'Thunderful Games': <Game_type.INDIE: 2>, 'Dear Villagers': <Game_type.INDIE: 2>, 'Twin Sails Interactive': <Game_type.INDIE: 2>, 'Dotemu': <Game_type.INDIE: 2>, '3D Realms': <Game_type.INDIE: 2>, 'Hooded Horse': <Game_type.INDIE: 2>, 'Paradox Arc': <Game_type.INDIE: 

This list of publishers is a good place to start, although it couldn't hurt to find more lists to add.

Next, apply this list of publishers to the database we're building. Only run the cell directly below this once or it will mess up the fields in the database.

In [10]:
GAME_TYPE = 16

original_fields.append("Game Type")

In [11]:
# Helper function to parse TEAM string
def parse_team_string(input):
    # First remove the brackets
    input = input[1:-1]
    # Split into two if there are multiple teams mentioned
    input = input.split(', ')
    # Remove the beginning and ending apostraphes from each team
    i = 0
    while i < len(input):
        input[i] = input[i][1:-1]
        i += 1
    return input

# Go through every row in the database and tag it with AAA, INDIE, or UNKNOWN
for row in original_rows:
    team = parse_team_string(row[TEAM]) # this is a string which may list both the dev and pub, or only one
    if row[PUBLISHER] != None:
        team.append(row[PUBLISHER])

    # Now team is a list from 1-3 of (possibly redundant) publishers. Run those publishers through the dict to classify the game
    game_type = None
    for pub in team:
        if pub in publisher_type_dict:
            game_type = publisher_type_dict[pub]
        else:
            game_type = Game_type.UNKNOWN
        
    row.append(game_type)


In [12]:
# Check how many classified games we have
num_classified = 0
for row in original_rows:
    if row[GAME_TYPE] != Game_type.UNKNOWN:
        num_classified += 1
    else:
        if row[PUBLISHER] != None:
            # This will help us decide if there are any repeating publishers that we should try to classify
            print(row[PUBLISHER])

print(num_classified)

miHoYo
Finji
Square
VU Games
Sierra Entertainment
THQ
DotEmu
id Software
Psyonix
Humble Bundle
Humble Bundle
Unknown
Playdead
Square
Spike
Square
Square
Motion Twin
PopCap Games
Focus Home Interactive
Galactic Cafe
Marvelous
Unknown Worlds Entertainment
Eidos Interactive
Unknown
Riot Games
IO Interactive
Unknown
WSS playground
Square EA
Frictional Games
Thunder Lotus Games
Delightworks
miHoYo
Finji
DotEmu
Majesco
id Software
Psyonix
Humble Bundle
Humble Bundle
Unknown
Focus Home Interactive
Galactic Cafe
Marvelous
Unknown Worlds Entertainment
Eidos Interactive
Unknown
Riot Games
The Behemoth
Unknown
IO Interactive
Unknown
WSS playground
Sierra Entertainment
Midway
Subset Games
Type-Moon
Binary Haze Interactive
Square EA
Frictional Games
Aksys Games
Hi-Rez Studios
Square-Enix
Ignition Entertainment
SNK
Rare
Microsoft
THQ
Unknown
THQ
Aqua Plus
Eidos Interactive
Neowiz Corporation
Square
Sierra Entertainment
Deep Silver
Square
Eidos Interactive
5pb
Unknown
Level 5
Inti Creates
Tecmo
Doubl

In [13]:
# Clean the dataset by removing all unclassified games
final_rows = []
for row in original_rows:
    if row[GAME_TYPE] != Game_type.UNKNOWN:
        final_rows.append(row)

In [14]:
# Helper function to parse GENRES string
def parse_genres_string(input):
    # First remove the brackets
    input = input[1:-1]
    # Split into multiple genres if there are multiple mentioned
    input = input.split(', ')
    # Remove the beginning and ending apostraphes from each genre
    i = 0
    while i < len(input):
        input[i] = input[i][1:-1]
        i += 1
    return input

# Gather info about genres. There are, at most, 6 genres per game. Most games have only 1-3 though
# Append info to new columns for ease of using Altair later
GENRE1 = 17
GENRE2 = 18
GENRE3 = 19
GENRE4 = 20
GENRE5 = 21
GENRE6 = 22
original_fields.append("Genre 1")
original_fields.append("Genre 2")
original_fields.append("Genre 3")
original_fields.append("Genre 4")
original_fields.append("Genre 5")
original_fields.append("Genre 6")

all_genres = []
max_genres_per_game = 0
for row in final_rows:
    current_genres = parse_genres_string(row[GENRES])
    
    # Update max_genres_per_game if necessary
    if len(current_genres) > max_genres_per_game:
        max_genres_per_game = len(current_genres)

    # Update list of all_genres if necessary
    for genre in current_genres:
        if genre not in all_genres:
            all_genres.append(genre)

    # Add genres to the row
    i = 0
    while i < 6:
        if len(current_genres) > i:
            # We have the i-th genre, so append it
            row.append(current_genres[i])
        else:
            row.append(None)
        i += 1
    
print("Max genres per game:", max_genres_per_game)
print("List of genres:", all_genres)    

Max genres per game: 6
List of genres: ['Adventure', 'RPG', 'Brawler', 'Indie', 'Turn Based Strategy', 'Platform', 'Simulator', 'Puzzle', 'Shooter', 'Music', 'Strategy', 'Fighting', 'Arcade', 'Visual Novel', 'Tactical', 'Racing', 'Point-and-Click', 'Sport', 'Card & Board Game', 'Real Time Strategy', '']


In [15]:
# Add data on sales and console
# Read in sales and console data
sales_and_console_fields = []
sales_and_console_rows = []
sales_and_console_dict = {}    # store TITLE -> INDEX, so that we can quickly find the correct row for any given game.
                               # We must use lists, because multiple games may be published under the same name (e.g. Tetris).

# reading csv file
with open("vgchartz-sales-and-consoles.csv", 'r', encoding='Latin1') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
 
    # extracting field names through first row
    sales_and_console_fields = next(csvreader)
 
    # extracting each data row one by one
    index = 0
    for row in csvreader:
        sales_and_console_rows.append(row)
        # Next, take care of the dictionary
        if row[2] in sales_and_console_dict:    # Check the titles
            # If there already is a value here, append the new value to the list
            sales_and_console_dict[row[2]].append(index)
        else:
            # If there isn't a value here yet, create a list for the dictionary
            sales_and_console_dict[row[2]] = [index]
            
        index += 1
 
    # get total number of rows
    print("Total no. of rows: %d" % (csvreader.line_num))

Total no. of rows: 57951


In [16]:
# Add the correct data from sales and consoles to the rest of our 
# EDIT: for now, only consider consoles, not sales.  The sales data has been incredibly hard to come by

import copy

TOTAL_SALES = 23
NA_SALES = 24
JP_SALES = 25
PAL_SALES = 26
OTHER_SALES = 27
CONSOLE = 28

my_sales_fields = original_fields.copy()
my_sales_rows = copy.deepcopy(final_rows)
#for row in final_rows:
#    my_sales_rows.append(final_rows.copy())
#    print(row)

my_sales_fields.append("Total Sales")
my_sales_fields.append("North American Sales")
my_sales_fields.append("Japan Sales")
my_sales_fields.append("PAL Sales")
my_sales_fields.append("Other Sales")
my_sales_fields.append("Console")

list_of_consoles = []

for row in my_sales_rows:
    # First, find the corresponding entry in sales and consoles
    title = row[TITLE]
    date = row[RELEASE_DATE]
    true_year = date[-2:]    # in format e.g. "17" or "23"

    # Check that it exists
    if title in sales_and_console_dict:
        # Once we get here, there is at least one entry in vgchartz for this title
        possible_indices = sales_and_console_dict[title]
        total_sales = None
        na_sales = None
        jp_sales = None
        pal_sales = None
        other_sales = None
        console = []

        # Pick out which of the possible indices is the correct one using the release date
        true_index = possible_indices
        found_info = False
        for index in possible_indices:
            possible_year = sales_and_console_rows[index][16][2:4]
            current_console = sales_and_console_rows[index][3]

            # If possible year is the same as true year, assume these are the same games. Check that this hit actually has the data we want, 
            # else ignore it because there are many repeat entries
            if possible_year == true_year and current_console != None:
                total_sales = sales_and_console_rows[index][11]
                na_sales = sales_and_console_rows[index][12]
                jp_sales = sales_and_console_rows[index][13]
                pal_sales = sales_and_console_rows[index][14]
                other_sales = sales_and_console_rows[index][15]
                console.append(current_console)
                #print(console)
                #print("current console", current_console)
                found_info = True
    
        # Append the correct information as found from vgchartz
        row.append(total_sales)
        row.append(na_sales)
        row.append(jp_sales)
        row.append(pal_sales)
        row.append(other_sales)
        if found_info == True:
            row.append(console)
            #print("console:", console)
            #print("row[CONSOLE]:", row[CONSOLE])
            #print("row", row)
        else:
            row.append(None)
        
    else:
        # There was no entry in vgchartz for this title, so we have no data
        row.append(None)    # TODO: this might not be the right syntax for an empty entry, double check this
        row.append(None)
        row.append(None)
        row.append(None)
        row.append(None)
        row.append(None)

In [17]:
# Finally, clean the dataset by removing rows from sales_final_rows if they don't have total sales data or console data
final_sales_rows = []
for row in my_sales_rows:
    if row[CONSOLE] != None:
        final_sales_rows.append(row)

print("Total no. of cleaned rows:", len(final_sales_rows))

Total no. of cleaned rows: 693


In [18]:
# Write to main CSV which will have data up to 2024
with open('compiled_db.csv', 'w', encoding='utf8', newline='') as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(original_fields)
    writer.writerows(final_rows)

In [19]:
# Write to sales and console CSV which will only have data up to 2020
with open('sales_compiled_db.csv', 'w', encoding='utf8', newline='') as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(my_sales_fields)
    writer.writerows(final_sales_rows)